The first step is to find reviewer ids that commonly occur together e.g. review the same hotels

In [19]:
import pandas as pd
import random
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

import statsmodels.api as sm
from scipy import stats

In [20]:
# connect to the sqlite database
import sqlite3

def connect_db(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

conn = connect_db("/Users/jamesfyfe/summer vac misc/yelpResData.db")


In [21]:
#### import the smaller subset data that has emotions of the actual review #########
df_emotion_and_LIWC = pd.read_csv("LIWC_and_emotions.csv")
#df_emotion_and_LIWC = pd.read_csv("LIWC_and_emotions.csv")
pos = (df_emotion_and_LIWC[df_emotion_and_LIWC["flagged"]=="Y"])
neg= (df_emotion_and_LIWC[df_emotion_and_LIWC["flagged"]=="N"])
df_neg_only = neg.sample(n=1500, random_state =1)
df_pos_only = pos.sample(n=1500, random_state =1)


df_emotion = pd.concat([df_neg_only, df_pos_only])
rev_id_list = list(df_emotion["reviewID"])

In [22]:
#### import the past reviews set and match it with the subset ##########
df_past = pd.read_csv("LIWC2015 Results (previous_reviews.csv).csv")
df_past.rename(columns={'K':'originalRevID','A': 'date', 'B': 'reviewID', 'C':'reviewerID', 'D': 'review', 'E': 'rating', 'F':'usefulCount', 'G':'coolCount','H':'funnyCount', 'I': 'flagged','J': 'restaurantID'}, inplace=True)
df_past=df_past.drop(0)
df_over_9 = df_past[df_past["WC"]>9]

df_over_9 = df_over_9[df_over_9["originalRevID"].isin(rev_id_list)]


/Users/jamesfyfe/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
'''import numpy as np
import random
np.random.seed(1)
conn.text_factory = lambda x: str(x, 'iso-8859-1')

# get one sample review to test the connection
def get_data(query):  
    cur = conn.cursor()
    # get all positives
    cur.execute(query)
    reviews = []
    rows = cur.fetchall()
    for row in rows:
        reviews.append(row)
    return reviews
#for row in rows:
#    print(row)
cur = conn.cursor()
cur.execute("SELECT * FROM review WHERE flagged LIKE 'Y' limit 1")
rows = cur.fetchall()

query_pos = "SELECT * FROM review WHERE flagged LIKE 'Y'" 
query_neg = "SELECT * FROM review WHERE flagged LIKE 'N'" 
# get review text
rows_pos = get_data(query_pos)
rows_neg = get_data(query_neg)

rows_neg = random.sample(rows_neg, len(rows_pos))
'''



In [13]:
'''reviews = (rows_pos + rows_neg)
random.shuffle(reviews)

#reviews = list(reviews)
for i in range (0, len(reviews)):
    reviews[i] = list(reviews[i])
    reviews[i][3] = reviews[i][3].replace('Â\xa0', '')'''

In [175]:
'''import csv

with open("YelpResReviews.csv", 'w') as file:
    wr = csv.writer(file, quoting=csv.QUOTE_ALL, delimiter=',')
    wr.writerow(["date","reviewID","reviewerID","reviewContent","rating","usefulCount","coolCount","funnyCount","flagged","restaurantID"])
    for i in range (0,len(reviews)):
        #for j in range (0,10):
        wr.writerow(reviews[i])
        #wr.writerow([x[j] for x in reviews[i]])'''

In [40]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, EmotionOptions, EntitiesOptions

authenticator = IAMAuthenticator('qK3W263cJ9IAmltdir-3IhfhEkb6Z5KyiDx-M9Jhm75P')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)

natural_language_understanding.set_service_url('https://api.au-syd.natural-language-understanding.watson.cloud.ibm.com/instances/49a00c0c-2c73-4ca1-bb57-b762514264e0')

'''response = natural_language_understanding.analyze(
    text=reviews[0][3],
    features=Features(
        entities=EntitiesOptions(emotion=True, limit=2)
    )).get_result()

print(json.dumps(response, indent=2))'''

'response = natural_language_understanding.analyze(\n    text=reviews[0][3],\n    features=Features(\n        entities=EntitiesOptions(emotion=True, limit=2)\n    )).get_result()\n\nprint(json.dumps(response, indent=2))'

In [48]:
#time_adj_emotion_df = []
#from ibm_watson import ApiException

i = 0
#exc = 0
for index, row in df_over_9.iterrows():
    if i > 29105:
        try:
            response = natural_language_understanding.analyze(
                text=row["review"],
                features=Features(emotion=EmotionOptions())).get_result()
            arr = []
            arr.append(response)
            arr.append(row["originalRevID"])
            arr.append(row["reviewID"])
            time_adj_emotion_df.append(arr)
                #print(i)
        except ApiException as ex:
            exc +=1
            pass
    i += 1

In [54]:
len(time_adj_emotion_df)

29110

In [96]:
time_adj_emotion_df

,emotions,reviewID,sadness,anger,disgust,joy,fear
0,"{'usage': {'text_units': 1, 'text_characters':...",ctA2PdCJlF0KeCBsP,0.292643,0.112365,0.088567,0.261127,0.109580
1,"{'usage': {'text_units': 1, 'text_characters':...",jwJTlc8qrBLpkYKt4CGmZg,0.542694,0.076944,0.616414,0.662569,0.069506
2,"{'usage': {'text_units': 1, 'text_characters':...",1mKFAuJueo4cJ-o0cG3ssA,0.025798,0.022255,0.006346,0.939005,0.008607
3,"{'usage': {'text_units': 1, 'text_characters':...",2J4AE24Wlu5XmeCLQP,0.157338,0.047234,0.079734,0.621915,0.065345
4,"{'usage': {'text_units': 1, 'text_characters':...",OELXVjEu2pxNAuzLJ47hvA,0.104872,0.046039,0.041377,0.731628,0.034957
...,...,...,...,...,...,...,...
29105,"{'usage': {'text_units': 1, 'text_characters':...",LNtM0Zg7QFwBq6kuLUcqUg,0.165529,0.116166,0.057548,0.691710,0.067401
29106,"{'usage': {'text_units': 1, 'text_characters':...",-2nm,0.425518,0.062322,0.073516,0.720362,0.068419
29107,"{'usage': {'text_units': 1, 'text_characters':...",5PiI8LiADzTjpLzUClz93Q,0.089564,0.087011,0.420219,0.688732,0.086729
29108,"{'usage': {'text_units': 1, 'text_characters':...",Ux9W4Y7w18lOtlc5igdTig,0.484980,0.067052,0.441654,0.671243,0.079364


In [60]:
len(rev_id_list)

3000

In [62]:
time_adj_emotion_df=pd.DataFrame(time_adj_emotion_df).rename(
    columns={0:"emotions",1:"originalRevID",2:"reviewID"}).drop(columns="originalRevID")
#df_temp = emotion_df
#df_over_9

In [106]:
#len(time_adj_emotion_df["reviewID"].unique())
#pd.merge(time_adj_emotion_df,df_over_9,on="reviewID")
#merged
#len(df_over_9["reviewID"].unique())
#len(df_over_9["reviewID"].unique())
df_over_9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29111 entries, 1 to 161337
Data columns (total 43 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           29111 non-null  object 
 1   reviewID       29110 non-null  object 
 2   reviewerID     29111 non-null  object 
 3   review         29111 non-null  object 
 4   rating         29111 non-null  object 
 5   usefulCount    29111 non-null  object 
 6   coolCount      29111 non-null  object 
 7   funnyCount     29111 non-null  object 
 8   flagged        29111 non-null  object 
 9   restaurantID   29111 non-null  object 
 10  originalRevID  29111 non-null  object 
 11  WC             29111 non-null  int64  
 12  Analytic       29111 non-null  float64
 13  Clout          29111 non-null  float64
 14  Authentic      29111 non-null  float64
 15  Tone           29111 non-null  float64
 16  Sixltr         29111 non-null  float64
 17  i              29111 non-null  float64
 18  we   

In [140]:
df_temp = time_adj_emotion_df
for i in range (0, len(df_temp)):
    df_temp.loc[i, 'sadness'] = df_temp["emotions"][i]["emotion"]["document"]["emotion"]["sadness"]
    df_temp.loc[i, 'anger'] = df_temp["emotions"][i]["emotion"]["document"]["emotion"]["anger"]
    df_temp.loc[i, 'disgust'] = df_temp["emotions"][i]["emotion"]["document"]["emotion"]["disgust"]
    df_temp.loc[i, 'joy'] = df_temp["emotions"][i]["emotion"]["document"]["emotion"]["joy"]
    df_temp.loc[i, 'fear'] = df_temp["emotions"][i]["emotion"]["document"]["emotion"]["fear"]

In [141]:
df_temp=df_temp.drop(columns="emotions").drop_duplicates()

In [142]:
#df_final = pd.merge(df_temp,df_emotion,on="reviewID")
#df_final = pd.merge(df_final,)
#df_final.info()

df_temp.rename(columns={"sadness":"priorSadness","anger":"priorAnger",
                        "fear":"priorFear","disgust":"priorDisgust",
                        "joy":"priorJoy"},inplace=True)

In [137]:
df_over_9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29111 entries, 1 to 161337
Data columns (total 43 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           29111 non-null  object 
 1   reviewID       29110 non-null  object 
 2   reviewerID     29111 non-null  object 
 3   review         29111 non-null  object 
 4   rating         29111 non-null  object 
 5   usefulCount    29111 non-null  object 
 6   coolCount      29111 non-null  object 
 7   funnyCount     29111 non-null  object 
 8   flagged        29111 non-null  object 
 9   restaurantID   29111 non-null  object 
 10  originalRevID  29111 non-null  object 
 11  WC             29111 non-null  int64  
 12  Analytic       29111 non-null  float64
 13  Clout          29111 non-null  float64
 14  Authentic      29111 non-null  float64
 15  Tone           29111 non-null  float64
 16  Sixltr         29111 non-null  float64
 17  i              29111 non-null  float64
 18  we   

In [139]:
#df_temp=df_temp.drop_duplicates()
pd.merge(df_over_9,df_temp, left_on='originalRevID', right_on='reviewID')


,date,reviewID_x,reviewerID,review,rating,usefulCount,coolCount,funnyCount,flagged,restaurantID,...,nonflu,filler,QMark,Exclam,reviewID_y,priorSadness,priorAnger,priorDisgust,priorJoy,priorFear
0,2009-02-26,zt,iplE0RwOqAHBumz5HwXiTQ,"One of my favorite hot dog places, and a place...",4,0,0,0,N,HOJqzz1WvOmeR9oESJ4d9A,...,0.00,0.00,0.00,0.00,L7jPyYvxxazyi2cm,0.692762,0.610781,0.069952,0.584238,0.685451
1,2009-02-26,ZynSNEGuPvGCj3eYIaLPlQ,EMQFO5mZFcLs347xAMdw4g,"Overall, it's a good place. Definitely not wor...",3,0,0,0,N,HOJqzz1WvOmeR9oESJ4d9A,...,0.00,0.00,0.00,0.69,L7jPyYvxxazyi2cm,0.692762,0.610781,0.069952,0.584238,0.685451
2,2009-02-26,WCInCRz,VVMS74JyUk2h53yfC-xNsA,I got there before the lunch rush and was amaz...,5,0,0,0,N,HOJqzz1WvOmeR9oESJ4d9A,...,0.00,0.00,0.00,2.15,L7jPyYvxxazyi2cm,0.692762,0.610781,0.069952,0.584238,0.685451
3,2009-02-26,nAj4AW4z-gqreF5JTWO4Qg,kxA0vTDi1KABuA5Tgg1HKA,"Include another in the ""me too"" chorus of love...",4,6,6,7,N,HOJqzz1WvOmeR9oESJ4d9A,...,1.78,0.00,1.33,0.44,L7jPyYvxxazyi2cm,0.692762,0.610781,0.069952,0.584238,0.685451
4,2009-02-23,sEbBzwrP5AdyGEfyNY00Pg,0P-3qUfy8sRcsUU_o2Kc-g,on saturday i stood in the freezing cold with ...,4,0,0,0,N,HOJqzz1WvOmeR9oESJ4d9A,...,0.00,0.00,0.00,1.68,L7jPyYvxxazyi2cm,0.692762,0.610781,0.069952,0.584238,0.685451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5581,2012-08-24,xzmgDv34U1OLvbRuHbLC5Q,WokddQRr1SF0V6gQZcIOvA,Let me be specific: I am not a pizza and beer ...,5,1,2,1,N,Z2nOMPedr7XYhgYhILJjEA,...,0.62,0.00,1.25,0.00,sFo8vXaisTNllBNsbnNmBQ,0.127347,0.081653,0.093651,0.633840,0.072033
5582,2012-08-24,Wqg2RXo7sk1pYV58,eEV5sZ_SpXNlG6jDJqFbEg,pizza is very good and the atmosphere is fun. ...,3,0,0,0,N,Z2nOMPedr7XYhgYhILJjEA,...,0.00,0.00,0.00,0.00,sFo8vXaisTNllBNsbnNmBQ,0.127347,0.081653,0.093651,0.633840,0.072033
5583,2012-08-23,Xw7tsdNJbpxpMKWlW5xJ3w,94cmXyxj9-TrtdlL3idleQ,Not sure what I can add that hasn't been said ...,4,7,7,4,N,Z2nOMPedr7XYhgYhILJjEA,...,0.00,0.00,0.00,0.00,sFo8vXaisTNllBNsbnNmBQ,0.127347,0.081653,0.093651,0.633840,0.072033
5584,2012-08-21,FXoREb0vdTkrbTbbx49YvA,hJl_bbjpXwBiDzKN_Vwtbw,Walked in and music was so loud we just turned...,2,0,0,0,N,Z2nOMPedr7XYhgYhILJjEA,...,0.00,3.03,0.00,0.00,sFo8vXaisTNllBNsbnNmBQ,0.127347,0.081653,0.093651,0.633840,0.072033


In [129]:
#pd.merge(df_final, df_emotion, left_on='originalRevID', right_on='reviewID')

#len(df_final["originalRevID"])
#len(df_emotion["reviewID"].unique())
#df_final.drop_duplicates()

,reviewID,priorSadness,priorAnger,priorDisgust,priorJoy,priorFear,date,reviewerID,review,rating,...,see,hear,feel,swear,netspeak,assent,nonflu,filler,QMark,Exclam
0,ctA2PdCJlF0KeCBsP,0.292643,0.112365,0.088567,0.261127,0.109580,2009-12-01,EM9qQxKzCbPONjOXbCCQBw,Cheap. Thats the only upside. After living in ...,2,...,0.00,0.00,0.00,0.0,3.45,0.00,0.00,0.0,0.00,0.00
1,jwJTlc8qrBLpkYKt4CGmZg,0.542694,0.076944,0.616414,0.662569,0.069506,2009-11-28,Gxu8g6IDK5pbM_9ReT8Hmg,Amazing. The most perfect slice of deep dish I...,5,...,0.90,0.00,0.90,0.0,0.00,0.00,0.90,0.0,0.00,0.00
2,1mKFAuJueo4cJ-o0cG3ssA,0.025798,0.022255,0.006346,0.939005,0.008607,2009-11-22,EQGNWIjSITjGXXWPCUGkDA,"Born and raised in Chicago, and god only knows...",4,...,0.00,0.00,0.00,0.0,1.67,0.00,0.00,0.0,0.00,1.67
3,2J4AE24Wlu5XmeCLQP,0.157338,0.047234,0.079734,0.621915,0.065345,2009-11-20,MOhA6cDlZ5hRHLuA_WOaDQ,"Yes, Ive indeed tried a lot of pizza. Ive had ...",4,...,1.35,1.35,0.00,0.0,0.00,2.03,0.00,0.0,0.68,0.00
4,OELXVjEu2pxNAuzLJ47hvA,0.104872,0.046039,0.041377,0.731628,0.034957,2009-11-14,7L2R2hs_VoT2rprrfizUHg,"Yum, I am always in a food coma after eating h...",4,...,0.00,0.00,3.45,0.0,0.00,0.00,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29100,LNtM0Zg7QFwBq6kuLUcqUg,0.165529,0.116166,0.057548,0.691710,0.067401,2009-12-04,BkZWHgKCwNc-9aJoLF0mUQ,"Warm it up, Kris...Im about to.... I have been...",4,...,1.33,0.00,1.33,0.0,0.00,0.00,0.67,0.0,0.00,0.00
29101,-2nm,0.425518,0.062322,0.073516,0.720362,0.068419,2009-11-20,AftFelF-uWV83h4nKTFawQ,This is not like Fogo or texas de brasil. It h...,3,...,0.78,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00
29102,5PiI8LiADzTjpLzUClz93Q,0.089564,0.087011,0.420219,0.688732,0.086729,2009-11-20,OCTep1VhmZ05l6FpiVhuUg,"I love this place. As far as rooftop bars go, ...",4,...,1.48,0.74,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00
29103,Ux9W4Y7w18lOtlc5igdTig,0.484980,0.067052,0.441654,0.671243,0.079364,2009-11-16,QOm-8-jt181pIoYPfJXMeA,Zed 451 is a cool hip looking placed but if yo...,3,...,1.97,0.79,0.79,0.0,0.00,0.79,0.00,0.0,0.00,0.39


In [88]:
df_final.rename(columns={"anger":"anger_LIWC","anger_x":"anger"},inplace=True)
df_final

,reviewID,sadness,anger,disgust,joy,fear,date,reviewerID,review,rating,...,see,hear,feel,swear,netspeak,assent,nonflu,filler,QMark,Exclam
0,ctA2PdCJlF0KeCBsP,0.292643,0.112365,0.088567,0.261127,0.109580,2009-12-01,EM9qQxKzCbPONjOXbCCQBw,Cheap. Thats the only upside. After living in ...,2,...,0.00,0.00,0.00,0.0,3.45,0.00,0.00,0.0,0.00,0.00
1,jwJTlc8qrBLpkYKt4CGmZg,0.542694,0.076944,0.616414,0.662569,0.069506,2009-11-28,Gxu8g6IDK5pbM_9ReT8Hmg,Amazing. The most perfect slice of deep dish I...,5,...,0.90,0.00,0.90,0.0,0.00,0.00,0.90,0.0,0.00,0.00
2,1mKFAuJueo4cJ-o0cG3ssA,0.025798,0.022255,0.006346,0.939005,0.008607,2009-11-22,EQGNWIjSITjGXXWPCUGkDA,"Born and raised in Chicago, and god only knows...",4,...,0.00,0.00,0.00,0.0,1.67,0.00,0.00,0.0,0.00,1.67
3,2J4AE24Wlu5XmeCLQP,0.157338,0.047234,0.079734,0.621915,0.065345,2009-11-20,MOhA6cDlZ5hRHLuA_WOaDQ,"Yes, Ive indeed tried a lot of pizza. Ive had ...",4,...,1.35,1.35,0.00,0.0,0.00,2.03,0.00,0.0,0.68,0.00
4,OELXVjEu2pxNAuzLJ47hvA,0.104872,0.046039,0.041377,0.731628,0.034957,2009-11-14,7L2R2hs_VoT2rprrfizUHg,"Yum, I am always in a food coma after eating h...",4,...,0.00,0.00,3.45,0.0,0.00,0.00,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29100,LNtM0Zg7QFwBq6kuLUcqUg,0.165529,0.116166,0.057548,0.691710,0.067401,2009-12-04,BkZWHgKCwNc-9aJoLF0mUQ,"Warm it up, Kris...Im about to.... I have been...",4,...,1.33,0.00,1.33,0.0,0.00,0.00,0.67,0.0,0.00,0.00
29101,-2nm,0.425518,0.062322,0.073516,0.720362,0.068419,2009-11-20,AftFelF-uWV83h4nKTFawQ,This is not like Fogo or texas de brasil. It h...,3,...,0.78,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00
29102,5PiI8LiADzTjpLzUClz93Q,0.089564,0.087011,0.420219,0.688732,0.086729,2009-11-20,OCTep1VhmZ05l6FpiVhuUg,"I love this place. As far as rooftop bars go, ...",4,...,1.48,0.74,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00
29103,Ux9W4Y7w18lOtlc5igdTig,0.484980,0.067052,0.441654,0.671243,0.079364,2009-11-16,QOm-8-jt181pIoYPfJXMeA,Zed 451 is a cool hip looking placed but if yo...,3,...,1.97,0.79,0.79,0.0,0.00,0.79,0.00,0.0,0.00,0.39


In [89]:
#df_final.to_csv('time_adjusted_LIWC_and_emotions.csv', index=False)